<a href="https://colab.research.google.com/github/argonism/TsukurinagaraRL/blob/master/Zerokara_chap5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Brainのネットワークで、

self.model.add_module('fc3', nn.Linear(32, num_actions))

の'fc3'を'fc2'としたばっかりに学習が進まず、
かつどこが間違っているのかわかりづらい沼にハマった。

# 深層強化学習DQN

ニューラルネットワークへの入力は、状態の各変数。
出力は$Q(s_t,a_t)$の値。つまり割引報酬和

誤差関数に、Q学習のアルゴリズムを使って学習する。

Q学習では、

$$Q(s_t, a_t) = R_{t+1} + \gamma \max_a Q(s_{t+1}, a)$$

これが成り立って欲しい。

そのため、$Q(s_t, a_t)$と$R_{t+1} + \gamma \max_a Q(s_{t+1}, a)$の二乗誤差を使う。

つまり
$$E(s_t, a_t) = (R_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t))^2$$


# 実装における注意

## experience replay
時間によって連続したステップの内容が似たものになった上で学習するのを防ぐために、ステップ内容は全てメモリに置いた上で、そのなかからランダムにピックアップして学習していく。

時間によって連続して似た内容のステップを学習するとパラメータが安定しないらしい。
わからん。

## Fixed Target Q-network
$Q(s_t, a)$の更新には、$Q(s_{t+1}, a)$が必要だが、ここで使用する関数Qが同じだとよろしくないらしく、学習最新版Q関数ではなく、少し前のものを使う。

更新学習が不安定になるらしい。わからん。

## 報酬のclipping
報酬は-1, 0, 1に固定する。
こうすることで、学習対象に依らずに同じハイパーパラメータでDQNを実行できるらしい。

わからん。

## Huber関数
誤差関数。二乗誤差より、誤差が大きい時の誤差eが抑えられる。

具体的には
- 値の差が-1 ~ 1の間なら、誤差eは二乗誤差と同じ
- それ以上の差なら、差の絶対値を取る。


ミニバッチの実装について注意してくれている。親切。



In [ ]:
# 使用するパッケージのインストール
# gym==0.17.2 pyvirtualdisplay==1.3.2
# xvfb=2:1.19.6-1ubuntu4.4 python-opengl=3.1.0+dfsg-1 ffmpeg=7:3.4.8-0ubuntu0.2
# JSAnimation==0.1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install JSAnimation > /dev/null 2>&1

In [ ]:
 import numpy as np
 import matplotlib.pyplot as plt
 %matplotlib inline
 import gym
from gym.wrappers import Monitor

In [ ]:
# 動画の描画関数の宣言
import glob
import io
import os
import base64
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython import display as ipythondisplay
from IPython.display import HTML
from pyvirtualdisplay import Display

display = Display(visible=0, size=(640, 400))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def reset_video():
  mp4list = glob.glob('video/*.mp4')
  for mp4 in mp4list:
    os.remove(mp4)

def wrap_env(env):
  env = Monitor(env, './video', force=True, video_callable=(lambda ep: ep % 100 == 0))
  reset_video()
  return env

In [ ]:
from collections import namedtuple
Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('名前Aです', 100)
print(Tr_object)
print(Tr_object.value_b)
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


ENV = 'CartPole-v0'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 500

tr(name_a='名前Aです', value_b=100)
100


In [ ]:
class ReplayMemory:
  def __init__(self, CAPACITY):
    self.capacity = CAPACITY
    self.memory = []
    self.index = 0
  
  def push(self, state, action, state_next, reward):
    if len(self.memory) < self.capacity:
      self.memory.append(None)
    
    self.memory[self.index] = Transition(state, action, state_next, reward)

    self.index = (self.index + 1) % self.capacity
  
  def sample(self, batch_size):
    return random.sample(self.memory, batch_size)
  
  def __len__(self):
    return len(self.memory)


In [ ]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

class Brain:
  def __init__(self, num_states, num_actions):
    self.num_actions = num_actions

    self.memory = ReplayMemory(CAPACITY)

    self.model = nn.Sequential()
    self.model.add_module('fc1', nn.Linear(num_states, 32))
    self.model.add_module('relu1', nn.ReLU())
    self.model.add_module('fc2', nn.Linear(32, 32))
    self.model.add_module('relu2', nn.ReLU())
    self.model.add_module('fc3', nn.Linear(32, num_actions))

    print(self.model)

    self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
  
  def replay(self):
    '''保存した行動や結果、状態から結合パラメータを学習する'''

    if len(self.memory) < BATCH_SIZE:
      return
    
    # ミニバッチの作成

    transitions = self.memory.sample(BATCH_SIZE)

    #transitions = (state, action, state_next, reward) * batch_size を　(state * batch_size, action * batch_size, state_next * batch_size, ...)に変える。
    batch = Transition(*zip(*transitions))

    # statem action , state_next, rewardごとにミニバッチに対応した形にする
    # stateなら (位置、速度、角度、角速度)の1*4のTensolがBATCH_SIZE分あったものを、BATCH_SIZE * 4のテンソルにする
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    # 教師信号となるQ(s_t, a_t)を求める

    self.model.eval()
    # self.model(state_batch) でbatch_size*2のテンソル(actionのQ値)が出る。
    # action_batchはbatch_size*(0, 1)のようになっている。
    # その後実行したactionの方のQ値を取り出すために、gatherを使っている。
    state_action_values = self.model(state_batch).gather(1, action_batch)

    # 次の状態(next_state)があるかどうかに注意しながら取り出し、max{Q(s_t+1, a)}を求める
    # ここではインデックスマスクを作る。...?
    # インデックスにByteTensor入れると、ByteTensorでTrueになってる要素を取り出して、そのそれぞれに対して代入を行えるっぽい。
    non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))

    next_state_values = torch.zeros(BATCH_SIZE)

    # 次の状態をモデルに入力して、次の行動のQ値が大きいものを取り出す。
    # それをnext_state_valuesの対応するインデックスに代入していく。
    # 後々微分されないために（固定するために）detach()をする。くわしくはわからん。
    # 現在の状態についてのQ値は更新対象であるから、detachしないっぽい。
    next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()

    # 教師となるQ値を求める。説明で書いた式の右辺。
    expected_state_action_values = reward_batch + GAMMA * next_state_values

    # 訓練

    self.model.train()

    # smooth_l1_lossはHuber関数
    # expected_state_action_valuesはsize: バッチサイズとなっているので、これをunsqueezeで(バッチサイズ*1)にする。...?
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze (1))
    
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

  def decide_action(self, state, episode):
    epsilon = 0.5 * (1 / (episode + 1))

    if epsilon <= np.random.uniform(0, 1):
      self.model.eval()
      with torch.no_grad():
        action = self.model(state).max(1)[1].view(1, 1)
        # view(1, 1)は、[torch.LongTensor of size 1]をsize 1x1にするらしい。逆にsize 1x1って何。

    else:
      action = torch.LongTensor([[random.randrange(self.num_actions)]])
    
    return action



In [ ]:
t = torch.zeros(10)
mask = t is not None
new = torch.tensor([[1, 0, 1, 1, 0, 1, 0, 0, 1, 1]])
print(t[mask])
t[mask] = new
print(t)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([1., 0., 1., 1., 0., 1., 0., 0., 1., 1.])


In [ ]:
class Agent:
  def __init__(self, num_states, num_actions):
    self.brain = Brain(num_states, num_actions)
  
  def update_q_function(self):
    self.brain.replay()
  
  def get_action(self, state, episode):
    action = self.brain.decide_action(state, episode)
    return action

  def memorize(self, state, action, state_next, reward):
    self.brain.memory.push(state, action, state_next, reward)

In [ ]:
class Environment:
  def __init__(self):
    self.env = wrap_env(gym.make(ENV))
    self.num_states = self.env.observation_space.shape[0]
    self.num_actions = self.env.action_space.n

    self.agent = Agent(self.num_states, self.num_actions)
  
  def run(self):
    complete_episodes = 0
    # 10回ごとに立ち続けたepisodeの平均を取る
    episode_10_list = np.zeros(10)
    # 連続成功記録
    complete_episodes = 0
    episode_final = False

    for episode in range(NUM_EPISODES):
      observation = self.env.reset()

      state = observation
      state = torch.from_numpy(state).type(torch.FloatTensor)
      state = torch.unsqueeze(state, 0)

      for step in range(MAX_STEPS):
        
        action = self.agent.get_action(state, episode)

        observation_next, _, done, _ = self.env.step(action.item())

        if done:
          state_next = None
          episode_10_list = np.hstack((episode_10_list[1:], step + 1))

          if step < 195:
            reward = torch.FloatTensor([-1.0])
            complete_episodes = 0
          else:
            reward = torch.FloatTensor([1.0])
            complete_episodes += 1
        else:
          reward = torch.FloatTensor([0.0])
          state_next = observation_next
          state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
          state_next = torch.unsqueeze(state_next, 0)
        
        self.agent.memorize(state, action, state_next, reward)

        self.agent.update_q_function()

        state = state_next

        if done:
          print(f'{episode} Episode: Finished after {step + 1} time steps: 10思考の平均step数 = {episode_10_list.mean()}')
          break
        
      if episode_final is True:
        show_video()
        break
      
      if complete_episodes >= 10:
        print('10回連続成功')
        episode_final = True

In [ ]:
cartpole_env = Environment()
cartpole_env.run()

Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 11 time steps: 10思考の平均step数 = 1.1
1 Episode: Finished after 9 time steps: 10思考の平均step数 = 2.0
2 Episode: Finished after 9 time steps: 10思考の平均step数 = 2.9
3 Episode: Finished after 8 time steps: 10思考の平均step数 = 3.7
4 Episode: Finished after 9 time steps: 10思考の平均step数 = 4.6
5 Episode: Finished after 13 time steps: 10思考の平均step数 = 5.9
6 Episode: Finished after 11 time steps: 10思考の平均step数 = 7.0
7 Episode: Finished after 9 time steps: 10思考の平均step数 = 7.9
8 Episode: Finished after 15 time steps: 10思考の平均step数 = 9.4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


9 Episode: Finished after 10 time steps: 10思考の平均step数 = 10.4
10 Episode: Finished after 15 time steps: 10思考の平均step数 = 10.8
11 Episode: Finished after 10 time steps: 10思考の平均step数 = 10.9
12 Episode: Finished after 12 time steps: 10思考の平均step数 = 11.2
13 Episode: Finished after 13 time steps: 10思考の平均step数 = 11.7
14 Episode: Finished after 14 time steps: 10思考の平均step数 = 12.2
15 Episode: Finished after 20 time steps: 10思考の平均step数 = 12.9
16 Episode: Finished after 18 time steps: 10思考の平均step数 = 13.6
17 Episode: Finished after 60 time steps: 10思考の平均step数 = 18.7
18 Episode: Finished after 25 time steps: 10思考の平均step数 = 19.7
19 Episode: Finished after 15 time steps: 10思考の平均step数 = 20.2
20 Episode: Finished after 18 time steps: 10思考の平均step数 = 20.5
21 Episode: Finished after 11 time steps: 10思考の平均step数 = 20.6
22 Episode: Finished after 13 time steps: 10思考の平均step数 = 20.7
23 Episode: Finished after 9 time steps: 10思考の平均step数 = 20.3
24 Episode: Finished after 8 time steps: 10思考の平均step数 = 19.7
25 Episode:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


100 Episode: Finished after 200 time steps: 10思考の平均step数 = 162.5
101 Episode: Finished after 200 time steps: 10思考の平均step数 = 170.5
102 Episode: Finished after 200 time steps: 10思考の平均step数 = 171.7
103 Episode: Finished after 200 time steps: 10思考の平均step数 = 181.1
104 Episode: Finished after 200 time steps: 10思考の平均step数 = 190.7
105 Episode: Finished after 200 time steps: 10思考の平均step数 = 195.0
106 Episode: Finished after 200 time steps: 10思考の平均step数 = 195.0
107 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.5
108 Episode: Finished after 200 time steps: 10思考の平均step数 = 197.1
109 Episode: Finished after 200 time steps: 10思考の平均step数 = 200.0
10回連続成功
110 Episode: Finished after 200 time steps: 10思考の平均step数 = 200.0
